In [1]:
import json
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from credentials import *
import pandas as pd
import numpy as np

In [3]:
import sys

In [4]:
print(sys.version)
print(tweepy.__version__)

3.8.0 (tags/v3.8.0:fa919fd, Oct 14 2019, 19:37:50) [MSC v.1916 64 bit (AMD64)]
3.9.0


In [2]:
"""Establecer acceso a la API de Twitter."""
CONSUMER_KEY = "uvcMjAH4FJglTG8QS9zlpJC01"
CONSUMER_SECRET = 'Ed2c2FhXzpHFagbfTVb3gTnziNTQ3hDk0UDRiiraPfupO5At0s'
ACCESS_TOKEN = "1442475234913775619-hKJkC4Pp3N0ztkvGAue6niVGtra8YZ"
ACCESS_SECRET = "QqW0LjrlYr1YrkllT1RhMbi4LmUvARaecltG6HTzAobUf"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)
print("authenticated")

authenticated


In [3]:
#tampico_madero = [-97.87777, 18.28519, -96.87777, 24.28519, -97.83623, 18.27228, -96.83623, 24.27228]
cdmx =  [-99.12766,19.42847,-98.12766,20.42847,-99.06224,19.35529,-98.06224,20.35529]
#cdmx_puebla = [-99.12766,19.42847,-98.12766,20.42847,-99.06224,19.35529,-98.06224,20.35529]

CONVERTIR TWEETS

In [4]:
# In[]:
"""Definir la clase que permitirá el uso del Twitter Streaming API."""
#Twitter Streaming API nos permite descargar mensajes de twitter en tiempo real
class listener(tweepy.StreamListener):
    
    def __init__(self, numero_tweets):
        self.received_tweets_counter = 0
        self.max_number_tweets = numero_tweets
        #Directorio y nombre del archivo de texto donde guardaremos los tweets
        self.file = open('C:/Users/jdroj/Downloads/BigData/TweetsMap3.8/tw_cdmxitz.csv', 'a', encoding="UTF-8")
        super(listener, self).__init__()
    
    def on_data(self, data):
        if (self.received_tweets_counter < self.max_number_tweets):
            
            self.received_tweets_counter += 1
            #La API de Twitter devuelve datos en formato JSON,
            #asi que hay que decodificarlos.
            try:
                decoded = json.loads(data)
            except Exception as e:
                print(e)
                return True
            #No todos los usuarios tienen habilitada la opcion de geolocalizacion
            #Por ello hay que dar formato a cuando no este disponible
            if decoded.get('geo') is not None:                
                location = str(decoded.get('geo').get('coordinates'))
            else:
                location = '[,]'
            
            #Extraer los datos que nos interese de los tweets
            text = decoded['text'].replace('\n',' ')
            user = '@' + decoded.get('user').get('screen_name')
            created = decoded.get('created_at')
            
            #Escribir los tweets en el archivo de texto
            self.file.write(user + "|" + location + "|" + created + "|" + text + "\n")
            
            return True
        else:
            self.file.close()
            print('Done!')
            return False

    #def on_status(self, status):
        #print(status.text)
        
    def on_error(self, status):
        print(status)
        #Debido a que el uso de la API de twitter tiene un limite diario debemos
        #desconectarnos del stream cuando excedamos dicho limite
        if status == 420:
            print('status code 420: ' + status)
            self.file.close()
            #Retornando un False en on_error Conseguimos desconectarnos del Stream
            return False
        self.file.close()

In [20]:
if __name__ == '__main__':
    print('Starting...')
    #Crear un Stream, estableciendo el número de tweets que se desean guardar
    twitterStream = tweepy.Stream(auth, listener(numero_tweets = 200))
    #Iniciar un Stream (Puede tomar mucho tiempo en terminar)
    #Mas info. en: https://developer.twitter.com/en/docs/tweets/filter-realtime/guides/basic-stream-parameters
    twitterStream.filter(locations = cdmx)

Starting...


In [5]:
tweets_raw = pd.read_table('C:/Users/jdroj/Downloads/BigData/TweetsMap3.8/tw_cdmxitz.csv', header = None, iterator=True)
tweets_2 = pd.DataFrame()
tweets_raw

In [9]:
#stamp = Timestamp('1/1/2014 16:20', tz='America/Sao_Paulo')
#new_stamp = stamp.tz_convert('US/Eastern')

In [6]:
while 1:
    #Debido a que el archivo de texto de los tweets es demasiado grande
    #debe de ser procesado por chunks en lugar de cargar todo el archivo en memoria
    tweets = tweets_raw.get_chunk(1000) #1000 filas por chunk
    tweets.columns = ['tweets']
    tweets['len'] = tweets.tweets.apply(lambda x: len(x.split('|')))
    tweets[tweets.len < 4] = np.nan
    del tweets['len']
    tweets = tweets[tweets.tweets.notnull()]
    
    #Establecer las columnas y valores que tomarán
    tweets['user'] = tweets.tweets.apply(lambda x: x.split('|')[0])
    tweets['geo'] = tweets.tweets.apply(lambda x: x.split('|')[1])
    tweets['timestamp'] = tweets.tweets.apply(lambda x: x.split('|')[2])
    tweets['tweet'] = tweets.tweets.apply(lambda x: x.split('|')[3])
    tweets['lat'] = tweets.geo.apply(lambda x: x.split(',')[0].replace('[',''))
    tweets['lon'] = tweets.geo.apply(lambda x: x.split(',')[1].replace(']',''))
    del tweets['tweets']
    del tweets['geo']
    
    #Convertir las latitudes y longitudes de string a float
    tweets['lon'] = pd.to_numeric(tweets['lon'], downcast="float")
    tweets['lat'] = pd.to_numeric(tweets['lat'], downcast="float")
    
    #Cambiar la zona horaria de UTC a GMT-5
    tweets['timestamp'] = pd.to_datetime(tweets['timestamp'], utc = False)
    #tweets = tweets.set_index('timestamp').tz_convert('America/Mexico_City').reset_index()
    
    #Almacenar los tweets en el dataframe
    tweets_2 = tweets_2.append(tweets, ignore_index = True)
    
    #Guardar los tweets en un archivo CSV
    tweets.to_csv('C:/Users/jdroj/Downloads/BigData/TweetsMap3.8/tw_cdmxitz.csv', mode='a', header=False,index=False)


StopIteration: 

In [7]:
min_lon = -99.99
min_lat = 19.30
max_lon = -98.0
max_lat = 20.50

#Descartar las filas con latitud y longitud nulas
tweets_2 = tweets_2[(tweets_2.lat.notnull()) & (tweets_2.lon.notnull())]

#Filtrar los tweets que estén dentro de la zona que nos interese
tweets_2 = tweets_2[(tweets_2.lon >= min_lon) & (tweets_2.lon <= max_lon) &
(tweets_2.lat >= min_lat) & (tweets_2.lat <= max_lat)]

In [8]:
with open('heatmap-files/tweets_heatmap_cdmxizt','w') as file:
 file.write(tweets_2[['lat','lon']].to_string(header=False, index=False))

III_GraficarMapa.py

Este script permite graficar las coordenadas del archivo (tweets_heatmap_tam_mad) en un 
mapa, con ayuda del script heatmap.py

In [9]:
import PIL
import osmviz
from PIL import Image
from PIL import ImageDraw, ImageFont
from IPython.display import Image
from IPython import get_ipython
import numpy as np
import imageio
from colour import Color
import pandas as pd
from copy import deepcopy

In [11]:
get_ipython().system('python heatmap.py -o maps/map_tweets_cdmxizt.png --width 1920 -p heatmap-files/tweets_heatmap_cdmxizt -b black -P equirectangular --osm --osm_base http://tile.memomaps.de/tilegen --decay 0.8 -r 10 --zoom 0 --margin 15')


Fetching tiles: 100%|██████████| 63/63 [00:34<00:00,  1.82tile/s]


In [16]:
im = PIL.Image.open('maps/map_tweets_cdmxizt.png')
draw = ImageDraw.Draw(im)
font = ImageFont.truetype("fonts/ProductSans.ttf", 14)
draw.text((1020, 600),"CDMX, Mex.", fill = "black", font=font)
draw.text((1080, 620),"43 tweets 24/07/20", fill = "black", font=font)
im.save('maps/map_tweets_cdmxizt_01.png')


Definir un nuevo gradiente de color que sustituya los puntos amarillos que marca heatmap.py por defecto.

In [14]:
hsva_min = Color()
hsva_min.hex_l = '#008dcd'

hsva_max = Color()
hsva_max.hex_l = '#4dccff'

color_gradient = list(hsva_max.range_to(hsva_min,256))
alpha = np.arange(0,256)[::-1]

gradient = []
for i, color_point in enumerate(color_gradient):
    rgb = list(color_point.get_rgb())
    rgb = [int(e * 255) for e in rgb]
    rgb.append(alpha[i])
    gradient.append([rgb])
color_gradient = np.array(gradient)

width = 43
from copy import deepcopy

color_gradient_row = deepcopy(color_gradient)

for col in range(width-1):
    color_gradient = np.hstack((color_gradient, color_gradient_row))
    
imageio.imwrite('gradients/gradient_blue.png', color_gradient)

Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.


In [15]:
get_ipython().system('python heatmap.py -G gradients/gradient_blue.png -o maps/map_tweets_cdmxizt_02.png --width 1920 -p heatmap-files/tweets_heatmap_cdmxizt -b black -P equirectangular --osm --osm_base https://basemaps.cartocdn.com/rastertiles/dark_all --decay 0.8 -r 10 --zoom 0 --margin 15')


Fetching tiles: 100%|██████████| 63/63 [00:17<00:00,  3.60tile/s]


In [17]:
import PIL
import osmviz
from PIL import Image
from PIL import ImageDraw, ImageFont
from IPython.display import Image
from IPython import get_ipython
import numpy as np
import imageio
from colour import Color
from copy import deepcopy
import pandas as pd
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
import sklearn
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
import joblib
import langid
from langdetect import detect
import textblob

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jdroj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
pd.set_option('max_colwidth',1000)
try:
    general_tweets_corpus_train = pd.read_csv('TASS-Dataset/general-train-tagged.csv', encoding='utf-8')
except:
    from lxml import objectify
    xml = objectify.parse(open('TASS-Dataset/general-train-tagged.xml', encoding="UTF-8"))
    root = xml.getroot()
    general_tweets_corpus_train = pd.DataFrame(columns=('content', 'polarity', 'agreement'))
    tweets = root.getchildren()
    
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], 
                       [tweet.content.text, tweet.sentiments.polarity.value.text,
                        tweet.sentiments.polarity.type.text]))
        row_s = pd.Series(row)
        row_s.name = i
        general_tweets_corpus_train = general_tweets_corpus_train.append(row_s)
        general_tweets_corpus_train.to_csv('TASS-Dataset/general-train-tagged.csv', index=False, encoding='utf-8')

try:
    general_tweets_corpus_test = pd.read_csv('TASS-Dataset/general-test-tagged.csv', encoding='utf-8')
except:
    from lxml import objectify
    xml = objectify.parse(open('TASS-Dataset/general-test-tagged.xml', encoding="UTF-8"))
    root = xml.getroot()
    general_tweets_corpus_test = pd.DataFrame(columns=('content', 'polarity'))
    tweets = root.getchildren()
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [tweet.content.text, tweet.sentiments.polarity.value.text]))
        row_s = pd.Series(row)
        row_s.name = i
        general_tweets_corpus_test = general_tweets_corpus_test.append(row_s)
        general_tweets_corpus_test.to_csv('TASS-Dataset/general-test-tagged.csv', index=False, encoding='utf-8')
        
try:
    stompol_tweets_corpus_train = pd.read_csv('TASS-Dataset/stompol-train-tagged.csv', encoding='utf-8')
except:
    from lxml import objectify
    xml = objectify.parse(open('TASS-Dataset/stompol-train-tagged.xml', encoding = "UTF-8"))
    root = xml.getroot()
    stompol_tweets_corpus_train = pd.DataFrame(columns=('content', 'polarity'))
    tweets = root.getchildren()
    
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [' '.join(list(tweet.itertext())), tweet.sentiment.get('polarity')]))
        row_s = pd.Series(row)
        row_s.name = i
        stompol_tweets_corpus_train = stompol_tweets_corpus_train.append(row_s)
        stompol_tweets_corpus_train.to_csv('TASS-Dataset/stompol-train-tagged.csv', index=False, encoding='utf-8')

try:
    stompol_tweets_corpus_test = pd.read_csv('TASS-Dataset/stompol-test-tagged.csv', encoding='utf-8')
except:
    from lxml import objectify
    xml = objectify.parse(open('TASS-Dataset/stompol-test-tagged.xml', encoding = "UTF-8"))
    root = xml.getroot()
    stompol_tweets_corpus_test = pd.DataFrame(columns=('content', 'polarity'))
    tweets = root.getchildren()
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [' '.join(list(tweet.itertext())), tweet.sentiment.get('polarity')]))
        row_s = pd.Series(row)
        row_s.name = i
        stompol_tweets_corpus_test = stompol_tweets_corpus_test.append(row_s)
        stompol_tweets_corpus_test.to_csv('TASS-Dataset/stompol-test-tagged.csv', index=False, encoding='utf-8')

try:
    social_tweets_corpus_test = pd.read_csv('TASS-Dataset/socialtv-test-tagged.csv', encoding='utf-8')
except:
    from lxml import objectify
    xml = objectify.parse(open('TASS-Dataset/socialtv-test-tagged.xml', encoding = "UTF-8"))
    root = xml.getroot()
    social_tweets_corpus_test = pd.DataFrame(columns=('content', 'polarity'))
    tweets = root.getchildren()
    
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [' '.join(list(tweet.itertext())), tweet.sentiment.get('polarity')]))
        row_s = pd.Series(row)
        row_s.name = i
        social_tweets_corpus_test = social_tweets_corpus_test.append(row_s)
        social_tweets_corpus_test.to_csv('TASS-Dataset/socialtv-test-tagged.csv', index=False, encoding='utf-8')

try:
    social_tweets_corpus_train = pd.read_csv('TASS-Dataset/socialtv-train-tagged.csv', encoding='utf-8')
except:
    from lxml import objectify
    xml = objectify.parse(open('TASS-Dataset/socialtv-train-tagged.xml', encoding = "UTF-8"))
    root = xml.getroot()
    social_tweets_corpus_train = pd.DataFrame(columns=('content', 'polarity'))
    tweets = root.getchildren()
    
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [' '.join(list(tweet.itertext())), tweet.sentiment.get('polarity')]))
        row_s = pd.Series(row)
        row_s.name = i
        social_tweets_corpus_train = social_tweets_corpus_train.append(row_s)
        social_tweets_corpus_train.to_csv('TASS-Dataset/socialtv-train-tagged.csv', index=False, encoding='utf-8')


try:
    general_tweets_corpus_train_mx = pd.read_csv('TASS-Dataset/general-train-tagged-mx.csv', encoding='utf-8')
except:
    from lxml import objectify
    xml = objectify.parse(open('TASS-Dataset/general-train-tagged-mx.xml', encoding="UTF-8"))
    #sample tweet object
    root = xml.getroot()
    general_tweets_corpus_train_mx = pd.DataFrame(columns=('content', 'polarity'))
    tweets = root.getchildren()
    
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [tweet.content.text, tweet.sentiment.polarity.value.text]))
        row_s = pd.Series(row)
        row_s.name = i
        general_tweets_corpus_train_mx = general_tweets_corpus_train_mx.append(row_s)
        general_tweets_corpus_train_mx.to_csv('TASS-Dataset/general-train-tagged-mx.csv', index=False, encoding='utf-8')

try:
    general_tweets_corpus_test_mx = pd.read_csv('TASS-Dataset/general-test-tagged-mx.csv', encoding='utf-8')
except:
    from lxml import objectify
    xml = objectify.parse(open('TASS-Dataset/general-test-tagged-mx.xml', encoding="UTF-8"))
    root = xml.getroot()
    general_tweets_corpus_test_mx = pd.DataFrame(columns=('content', 'polarity'))
    tweets = root.getchildren()
    
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [tweet.content.text, tweet.sentiment.polarity.value.text]))
        row_s = pd.Series(row)
        row_s.name = i
        general_tweets_corpus_test_mx = general_tweets_corpus_test_mx.append(row_s)
        general_tweets_corpus_test_mx.to_csv('TASS-Dataset/general-test-tagged-mx.csv', index=False, encoding='utf-8')


In [19]:
tweets_corpus = pd.concat([
    social_tweets_corpus_train,
    social_tweets_corpus_test,
    stompol_tweets_corpus_test,
    stompol_tweets_corpus_train,
    general_tweets_corpus_test,
    general_tweets_corpus_train,
    general_tweets_corpus_test_mx,
    general_tweets_corpus_train_mx
])

In [20]:
tweets_corpus = tweets_corpus.query('agreement != "DISAGREEMENT" and polarity != "NONE"')

Definir las funciones para la tokenización y stemming

In [21]:
#Eliminar enlaces
tweets_corpus = tweets_corpus[-
tweets_corpus.content.str.contains('^http.*$')]
spanish_stopwords = stopwords.words('spanish')
non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))
stemmer = SnowballStemmer('spanish')

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # Eliminar caracteres que no sean letras
    text = ''.join([c for c in text if c not in non_words])
    # tokenize
    tokens = word_tokenize(text)
    # stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

Preparar el modelo que evaluará los tweets.
Agregar una nueva columna al data frame corpus (tweets_corpus) en donde las polaridades de los tweets serán representadas de forma binaria.

In [22]:
tweets_corpus = tweets_corpus[tweets_corpus.polarity != 'NEU']
tweets_corpus['polarity_bin'] = 0
tweets_corpus.polarity_bin[tweets_corpus.polarity.isin(['P', 'P+'])] = 1
tweets_corpus.polarity_bin.value_counts(normalize=True)

C:\Users\jdroj\AppData\Local\Temp/ipykernel_18608/548591396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_corpus.polarity_bin[tweets_corpus.polarity.isin(['P', 'P+'])] = 1


1    0.571094
0    0.428906
Name: polarity_bin, dtype: float64

In [23]:
vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = spanish_stopwords
)

pipeline = Pipeline([
    ('vect', vectorizer),
    ('cls', LinearSVC()),
])

parameters = {
    'vect__max_df': (0.5, 1.9),
    'vect__min_df': (10, 20,50),
    'vect__max_features': (500, 1000),
    'vect__ngram_range': ((1, 1), (1, 2)), # unigrams or bigrams
    'cls__C': (0.2, 0.5, 0.7),
    'cls__loss': ('hinge', 'squared_hinge'),
    'cls__max_iter': (500, 1000)
}


In [26]:
#grid_search = sklearn.model_selection.GridSearchCV(pipeline, parameters, n_jobs=-1 , scoring='roc_auc')
#grid_search.fit(tweets_corpus.content, tweets_corpus.polarity_bin)
#grid_search.best_params_

In [24]:
grid_search = joblib.load('grid_search.pkl')

c:\Users\jdroj\Downloads\BigData\TweetsMap3.8\env\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\jdroj\Downloads\BigData\TweetsMap3.8\env\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22.1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\jdroj\Downloads\BigData\TweetsMap3.8\env\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator Pipeline from version 0.22.1 when using ver

In [25]:
model = LinearSVC(C=.2, loss='squared_hinge',max_iter=1000,multi_class='ovr',
                  random_state=None,penalty='l2',tol=0.0001)

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = spanish_stopwords,
    min_df = 50,
    max_df = 0.9,
    ngram_range=(1, 1),
    max_features=1000
)

corpus_data_features = vectorizer.fit_transform(tweets_corpus.content)
corpus_data_features_nd = corpus_data_features.toarray()

scores = cross_val_score(
    model,
    corpus_data_features_nd[0:len(tweets_corpus)],
    y=tweets_corpus.polarity_bin,
    scoring='roc_auc',
    cv=5
)

scores.mean()

c:\Users\jdroj\Downloads\BigData\TweetsMap3.8\env\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['algun', 'com', 'contr', 'cuand', 'desd', 'dond', 'durant', 'eram', 'estab', 'estais', 'estam', 'estan', 'estand', 'estaran', 'estaras', 'esteis', 'estem', 'esten', 'estes', 'estuv', 'fuer', 'fues', 'fuim', 'fuist', 'hab', 'habr', 'habran', 'habras', 'hast', 'hem', 'hub', 'mas', 'mia', 'mias', 'mio', 'mios', 'much', 'nad', 'nosotr', 'nuestr', 'par', 'per', 'poc', 'porqu', 'qui', 'seais', 'seam', 'sent', 'ser', 'seran', 'seras', 'si', 'sient', 'sint', 'sobr', 'som', 'suy', 'tambien', 'tant', 'ten', 'tendr', 'tendran', 'tendras', 'teng', 'tien', 'tod', 'tuv', 'tuy', 'vosotr', 'vuestr'] not in stop_words.
  warnings.warn(


0.9168214861312085

Convertir el archivo CSV con los tweets (tweets_tampico_madero.csv) en un dataframe y seleccionar
aquellos que se encuentren dentro del área que nos interesa.

In [27]:
tweets = pd.read_csv('C:/Users/jdroj/Downloads/BigData/TweetsMap3.8/tw_cdmxitz.csv', 
                     names = ["timestamp", "user", "tweet", "lat", "lon"],encoding='utf-8')


tweets = tweets[tweets.tweet.str.len() < 150]

tweets.lat = pd.to_numeric(tweets.lat, errors='coerce')
tweets.lon = pd.to_numeric(tweets.lon, errors='coerce')

tweets = tweets[tweets.lat.notnull()]

min_lon = -99.12766
min_lat = 19.42847
max_lon = -98.12766
max_lat = 20.42847

tweets = tweets[(tweets.lat.notnull()) & (tweets.lon.notnull())]
tweets = tweets[(tweets.lon > min_lon) & (tweets.lon < max_lon) & (tweets.lat > min_lat) & (tweets.lat < max_lat)]


In [28]:
def langid_safe(tweet):
 try:
    return langid.classify(tweet)[0]
 except Exception as e:
    pass
 
def langdetect_safe(tweet):
 try:
    return detect(tweet)
 except Exception as e:
    pass

def textblob_safe(tweet):
 try:
    return textblob.TextBlob(tweet).detect_language()
 except Exception as e:
    pass
 
#Puede tomar un tiempo en terminar
tweets['lang_langid'] = tweets.tweet.apply(langid_safe)
tweets['lang_langdetect'] = tweets.tweet.apply(langdetect_safe)
tweets['lang_textblob'] = tweets.tweet.apply(textblob_safe)
tweets['lang_textblob'] = tweets.tweet.apply(textblob_safe)

Exportar el data frame de los tweets con el campo de identificación de idioma a un archivo CSV (tweets_cdmx.csv)

In [29]:
tweets.to_csv('heatmap-files/tw_cdmxizt_02.csv', encoding='utf-8')

Modificar el data frame de los tweets para solo guardar aquellos que estén en idioma español.

In [30]:
tweets = tweets.query(''' lang_langdetect == 'es' or lang_langid == 'es' or lang_textblob == 'es' ''')

Utilizar el modelo entrenado para predecir los sentimientos de lo tweets y exportar el data frame resultante a un 
archivo CSV (tweets_polarity_bin.csv) y a un archivo de texto (tweets_heatmap_polarity_binary) para ser graficado 
posteriormente.

In [31]:
pipeline = Pipeline([
    ('vect', CountVectorizer(
        analyzer = 'word',
        tokenizer = tokenize,
        lowercase = True,
        stop_words = spanish_stopwords,
        min_df = 50,
        max_df = 0.9,
        ngram_range=(1, 1),
        max_features=1000
    )),
    ('cls', LinearSVC(C=.2, loss='squared_hinge', max_iter=1000, multi_class='ovr',
        random_state=None,
        penalty='l2',
        tol=0.0001
    )),
])

pipeline.fit(tweets_corpus.content, tweets_corpus.polarity_bin)

tweets['polarity'] = pipeline.predict(tweets.tweet)
tweets[['tweet', 'lat', 'lon', 'polarity']].to_csv('heatmap-files/tweets_polarity_bin.csv', encoding='utf-8')

with open('heatmap-files/tweets_heatmap_polarity_binary','w') as file:
    file.write(tweets[['lat','lon', 'polarity']].to_string(header=False, index=False))

c:\Users\jdroj\Downloads\BigData\TweetsMap3.8\env\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['algun', 'com', 'contr', 'cuand', 'desd', 'dond', 'durant', 'eram', 'estab', 'estais', 'estam', 'estan', 'estand', 'estaran', 'estaras', 'esteis', 'estem', 'esten', 'estes', 'estuv', 'fuer', 'fues', 'fuim', 'fuist', 'hab', 'habr', 'habran', 'habras', 'hast', 'hem', 'hub', 'mas', 'mia', 'mias', 'mio', 'mios', 'much', 'nad', 'nosotr', 'nuestr', 'par', 'per', 'poc', 'porqu', 'qui', 'seais', 'seam', 'sent', 'ser', 'seran', 'seras', 'si', 'sient', 'sint', 'sobr', 'som', 'suy', 'tambien', 'tant', 'ten', 'tendr', 'tendran', 'tendras', 'teng', 'tien', 'tod', 'tuv', 'tuy', 'vosotr', 'vuestr'] not in stop_words.
  warnings.warn(


In [41]:
# Verde = tweets positivos
hsva_min = Color()
hsva_min.hex_l = '#24b736'
hsva_max = Color()
hsva_max.hex_l = '#24b736'
color_gradient = list(hsva_max.range_to(hsva_min,256))
alpha = np.arange(0,256)[::-1]
gradient = []

for i, color_point in enumerate(color_gradient):
    rgb = list(color_point.get_rgb())
    rgb = [int(e * 255) for e in rgb]
    rgb.append(alpha[i])
    gradient.append([rgb])

color_gradient = np.array(gradient)

width = 43
color_gradient_row = deepcopy(color_gradient)

for col in range(width-1):
    color_gradient = np.hstack((color_gradient, color_gradient_row))

imageio.imwrite('gradients/gradient_green.png', color_gradient)

# Rojo = tweets negativos
hsva_min = Color()
hsva_min.hex_l = '#ff3639'

hsva_max = Color()
hsva_max.hex_l = '#ff3639'

color_gradient = list(hsva_max.range_to(hsva_min,256))
alpha = np.arange(0,256)[::-1]

gradient = []
for i, color_point in enumerate(color_gradient):
    rgb = list(color_point.get_rgb())
    rgb = [int(e * 255) for e in rgb]
    rgb.append(alpha[i])
    gradient.append([rgb])

color_gradient = np.array(gradient)
width = 43

color_gradient_row = deepcopy(color_gradient)

for col in range(width-1):
    color_gradient = np.hstack((color_gradient, color_gradient_row))

imageio.imwrite('gradients/gradient_red.png', color_gradient)

Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.


In [32]:
tweets = pd.read_table('heatmap-files/tweets_heatmap_polarity_binary', encoding='utf-8', sep=' ', header=None)
tweets

,0,1,2
0,19.470146,-99.125560,1
1,19.457650,-99.069330,1
2,20.157000,-98.189760,0
3,19.531414,-98.858920,0
4,19.511335,-99.041310,1
5,19.458115,-99.069190,0
6,19.511335,-99.041310,1
7,19.511335,-99.041310,1
8,19.436243,-99.069560,1
9,19.693474,-98.847810,1


In [33]:
#del tweets[tweets.columns[2]]
#del tweets[tweets.columns[0]]
tweets
tweets.columns = ['lat', 'lon', 'polarity']

In [34]:
with open('heatmap-files/tweets_with_polarity_negative','w') as file:
    file.write(tweets[['lat','lon']][tweets.polarity==0].to_string(header=False, index=False))


In [35]:
with open('tweets_with_polarity_positive','w') as file:
    file.write(tweets[['lat','lon']][tweets.polarity==1].to_string(header=False, index=False))

In [36]:
get_ipython().system('python heatmap.py -o maps/cdmxizt_tweets_polarity_negative.png --width 1920 -p heatmap-files/tweets_with_polarity_negative -P equirectangular --osm --osm_base https://basemaps.cartocdn.com/rastertiles/light_all --decay 0.8 -r 10 --zoom 0 --margin 15 -G gradients/gradient_red.png --layer')


Fetching tiles: 100%|██████████| 36/36 [00:06<00:00,  5.42tile/s]


In [37]:
get_ipython().system('python heatmap.py -o maps/cdmxizt_tweets_polarity_positive.png --width 1920 -p heatmap-files/tweets_with_polarity_positive -P equirectangular --osm --osm_base https://basemaps.cartocdn.com/rastertiles/light_all --decay 0.8 -r 10 --zoom 0 --margin 15 -G gradients/gradient_green.png')


Fetching tiles: 100%|██████████| 18/18 [00:00<00:00, 488.44tile/s]


In [39]:
background = PIL.Image.open('maps/cdmxizt_tweets_polarity_positive.png')
foreground = PIL.Image.open('maps/cdmxizt_tweets_polarity_negative.png')

background.paste(foreground, (0, 0), foreground)
draw = ImageDraw.Draw(background)
font = ImageFont.truetype("fonts/ProductSans.ttf", 14)

draw.text((1020, 600),"CDMX, Mex.", fill = "black", font=font)
draw.text((1020, 620),"Tweets positivos y negativos", fill = "black", font=font)

background.save('maps/cdmxizt_tweets_polarity.png')